In [13]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm
import intake
import fsspec
import seaborn as sns
#import gcsfs
import cftime
import pandas as pd
import xesmf as xe
#import xskillscoere as xs
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cartopy as cart
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from shapely.geometry import Point

In [14]:
chicago_LAT=41.88
chicago_LON=(360-87.6298)%360
#
cvals = '/Users/hrh/Desktop/TwoMoments21/cvals/cmip6/'

In [15]:
# calculate global means

def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat        = ds[get_lat_name(ds)]
    weight     = np.cos(np.deg2rad(lat))
    weight    /= weight.mean()
    other_dims = set(ds.dims) - {'quantile'}
    return (ds * weight).mean(other_dims)

In [16]:
final_ds_pi  = xr.open_zarr(cvals+'cmip6_pi_quantiles_annual.zarr').tasmax
final_ds_eoc = xr.open_zarr(cvals+'cmip6_eoc_quantiles_annual.zarr').tasmax
final_ds_pi

<xarray.DataArray 'tasmax' (source_id: 18, year: 30, dayofyear: 365, lat: 61,
                            lon: 121)>
dask.array<open_dataset-tasmax, shape=(18, 30, 365, 61, 121), dtype=float32, chunksize=(3, 8, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * year       (year) int64 1850 1851 1852 1853 1854 ... 1876 1877 1878 1879

In [17]:
################################################### Land and Ocean -only curves #######################

In [18]:
    def is_land(lon, lat):
        # Create a Natural Earth Reader
        land_shp_fname = shpreader.natural_earth(resolution='110m', category='physical', name='land')
        # Load the shapefile using Shapely's shape() function
        land_geom = [geom for geom in shpreader.Reader(land_shp_fname).geometries()]
        # Check if the point is within any land geometry
        return any([geom.contains(Point(lon, lat)) for geom in land_geom])

In [19]:
def create_land_mask(ds):
    """Create a land mask for a given xarray dataset."""
    
    # Get the latitude and longitude points from the dataset
    lats = ds['lat'].values
    lons = ds['lon'].values

    # Create an empty mask with the same shape as the data
    land_mask = np.empty((len(lats), len(lons)))

    # Fill the mask using the is_land function
    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            land_mask[i, j] = is_land(lon, lat)

    # Create a new DataArray with the land mask
    ds_land_mask = xr.DataArray(land_mask, coords=[lats, lons], dims=['lat', 'lon'])

    # Add the land mask to the dataset and return
    ds['land_mask'] = ds_land_mask
    return ds

In [20]:
%%time
final_pi_mask  = create_land_mask(final_ds_pi)
final_eoc_mask = create_land_mask(final_ds_eoc)
final_eoc_mask

CPU times: user 1min 48s, sys: 1.41 s, total: 1min 49s
Wall time: 1min 50s


<xarray.DataArray 'tasmax' (source_id: 18, year: 30, dayofyear: 365, lat: 61,
                            lon: 121)>
dask.array<open_dataset-tasmax, shape=(18, 30, 365, 61, 121), dtype=float32, chunksize=(3, 8, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
    land_mask  (lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0

In [ ]:
land_temperature = ds['temperature'] * ds['land_mask']